In [15]:
%matplotlib inline

In [1]:
import keras
import pandas as pd
import numpy as np
import glob

from utils.schemas import schema_train_4

Using TensorFlow backend.


# Load Data

Para cargar los datos con pandas, voy a usar la misma forma que usamos en LightGBM

In [4]:
!ls ../../../data/

data_4.zip  df_num    test_final_4  train_final_4
df_cat	    test.csv  train.csv     train_model_little.csv


In [80]:
RUTA_TRAIN = "../../../data/train_final_4/"
RUTA_TEST = "../../../data/test_final_4/"

In [86]:
# print('Cargando datos del TRAIN')
# path = RUTA_TRAIN
# allFiles = glob.glob(path + "/*.csv")
# list_ = []
# for file_ in allFiles:
#     df = pd.read_csv(file_)
#     df = (df.fillna(-1)).astype(schema_train_4)
#     list_.append(df)

# train = pd.concat(list_, axis = 0, ignore_index= True)


In [9]:
df = pd.read_csv(RUTA_TRAIN)
# df = (df.fillna(0)).astype(schema_train_4)
df = (df).astype(schema_train_4)


#train = df
# train.head()

In [51]:
categoric_features = ['ProductName_index','EngineVersion_0_index','EngineVersion_1_index','Census_OSVersion_0_index','Census_OSVersion_1_index','AppVersion_0_index','AppVersion_1_index','AvSigVersion_0_index','AvSigVersion_1_index','OsVer_0_index','OsVer_1_index','Platform_index','Processor_index','OsPlatformSubRelease_index','OsBuildLab_index','SkuEdition_index','PuaMode_index','SmartScreen_index','Census_MDC2FormFactor_index','Census_DeviceFamily_index','Census_ProcessorClass_index','Census_PrimaryDiskTypeName_index','Census_ChassisTypeName_index','Census_PowerPlatformRoleName_index','Census_InternalBatteryType_index','Census_OSArchitecture_index','Census_OSBranch_index','Census_OSEdition_index','Census_OSSkuName_index','Census_OSInstallTypeName_index','Census_OSWUAutoUpdateOptionsName_index','Census_GenuineStateName_index','Census_ActivationChannel_index','Census_FlightRing_index','OsVer_index','DefaultBrowsersIdentifier','AVProductStatesIdentifier','CountryIdentifier','CityIdentifier','OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','IeVerIdentifier','Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier','Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Wdft_RegionIdentifier','RtpStateBitfield','prediction_2','prediction_4','prediction_8','prediction_16','prediction_32','prediction_64']

booleans_features = ['Census_IsSecureBootEnabled','Census_IsWIMBootEnabled','Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable','Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Census_IsPortableOperatingSystem','Census_IsFlightingInternal','Census_IsFlightsDisabled','IsProtected','IsBeta','IsSxsPassiveMode','AVProductsInstalled','AVProductsEnabled','HasTpm','Census_HasOpticalDiskDrive','Census_ThresholdOptIn','Census_InternalBatteryType_informed','Firewall','SMode','AutoSampleOptIn']

numerics_features = ['OsBuild','OsSuite','UacLuaenable','Census_ProcessorCoreCount','Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_InternalBatteryNumberOfCharges','Census_OSBuildNumber','Census_OSBuildRevision','OsBuildLab_diff','std_diff_DateOsBuildLab','AvSigVersion_diff','std_diff_AvSigVersion','OSVersion_diff','std_diff_OSVersion','max_OsBuildLab_diff','max_AvSigVersion_diff','max_OSVersion_diff','ratio_OsBuildLab_diff','ratio_AvSigVersion_diff','ratio_OSVersion_diff','count(DISTINCT AvSigVersion_Name)','count(DISTINCT AvSigVersion_Type)','count(DISTINCT AvSigVersion_AlertLevel)','count1','count2','count3','count4','count5','count6','count7','count8']

__FOR TEST ONLY__

Select only features with < 100 categories

In [29]:
categorical_features_selected = []

for i in categoric_features:
    if df[i].nunique() < 100:
        categorical_features_selected.append(i)

Change -1 values by 99

In [36]:
columns_bad = []
for i in categorical_features_selected:
    if(len(df[df[i] == -1].values)) > 0:
        print(i)
        columns_bad.append(i)
    
        

for i in columns_bad:
     df.loc[df[i] == -1,i] = 99
    

OrganizationIdentifier
RtpStateBitfield


Change df to train

In [43]:
train = df

## Generate X and Y 

Remove cols that not improve the lgbm model, and MachineIdentifier, and HasDetection

In [44]:
drop_version = ['AvSigVersion_index', 'EngineVersion_index', 'Census_OSVersion_index', 'AppVersion_index']

sel_cols = [c for c in train.columns if c not in ['MachineIdentifier',
                                                      'HasDetections',
                                                      'Census_DeviceFamily_Windows.Server',
                                                      'Census_DeviceFamily_Windows.Desktop'
                                                     ]+drop_version]

In [45]:
X = train.loc[:, sel_cols]
y = train.loc[:,'HasDetections']

## Train y test

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [48]:
print("X_train :"+str(X_train.shape))
print("X_test :"+str(X_test.shape))
print("y_train :"+str(y_train.shape))
print("y_test :"+str(y_test.shape))

X_train :(89188, 117)
X_test :(22298, 117)
y_train :(89188,)
y_test :(22298,)


# Keras Model

In [49]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Embedding,Input,Concatenate,Reshape

## Metrics

### AUC 1

Easy

In [ ]:
from sklearn.metrics import roc_auc_score
import tensorflow as tf

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

### AUC 2

For Keras.

A little more complex

In [ ]:
from keras import callbacks
from sklearn.metrics import roc_auc_score

class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return

## EarlyStopping

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='auroc', patience=20, verbose=2,min_delta=0.01, mode='auto')

# Neural Network



## Embedding Layers

This is a NN example. I got from Kaggle

```py
input_numeric = Input(shape=(24,))
embedding_numeric = Dense(16)(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)

x = Concatenate()(embeddings)
x = Dense(80, activation='relu')(x)
x = Dropout(.35)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(.15)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(.15)(x)
output = Dense(1, activation='sigmoid')(x)
```

In [66]:
ins = []
outs = []
for i in categorical_features_selected:
    x = Input(shape=(1,),name='input_'+i)
    ins.append(x)
    x = Embedding(df[i].max()+1, df[i].max()+1, input_length=1,name='embedding_'+i)(x)
    x = Reshape(name='reshape_'+i,target_shape=(df[i].max()+1,))(x)
    outs.append(x)
    
# for i in booleanas:
#     x = Input(shape=(1,))
#     ins.append(x)   
#     x = Embedding(df[i].nunique(), df[i].nunique(), input_length=1)(x)
#     x = Reshape(target_shape=(df[i].nunique(),))(x)
#     outs.append(x)

## Network Architecture

In [67]:
primer_input_len = len(numericas+booleanas)
primer_input_len

58

In [68]:
len_batch_size = int(X_train.shape[0]/1000)
epochs_to_train = 5

In [69]:
cat_inp = Input(shape=(primer_input_len, ),name="Categoricas_Booleanas")
ins.append(cat_inp)
# x = Dense(50)(x) 
# outs.append(x)

x = Concatenate()(outs)

x = Concatenate()([x,cat_inp])

x = Dense(256, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(.1)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=ins, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['acc'])

### Save model

In [71]:
model.save("modelo3.h5")

## FIT!

In [73]:
model.fit( [X_train[col] for col in categoricas_selected] +  [X_train[numericas+booleanas]], 
           y_train,
          epochs=epochs_to_train,
           validation_data = ([X_test[col] for col in categoricas_selected] +  [X_test[numericas+booleanas]],y_test))

Train on 89188 samples, validate on 22298 samples
Epoch 1/5
89188/89188 [==============================] - 21s 231us/step - loss: 8.0491 - acc: 0.4993 - val_loss: 8.0771 - val_acc: 0.4989
Epoch 2/5
89188/89188 [==============================] - 18s 204us/step - loss: 8.0585 - acc: 0.5000 - val_loss: 8.0771 - val_acc: 0.4989
Epoch 3/5
89188/89188 [==============================] - 18s 204us/step - loss: 8.0585 - acc: 0.5000 - val_loss: 8.0771 - val_acc: 0.4989
Epoch 4/5
89188/89188 [==============================] - 18s 204us/step - loss: 8.0585 - acc: 0.5000 - val_loss: 8.0771 - val_acc: 0.4989
Epoch 5/5
89188/89188 [==============================] - 18s 204us/step - loss: 8.0585 - acc: 0.5000 - val_loss: 8.0771 - val_acc: 0.4989


In [44]:
from ann_visualizer.visualize import ann_viz

ann_viz(model, title="NN sin densidad")

ValueError: ANN Visualizer: Layer not supported for visualizing

# Graphs

In [ ]:
model.history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model.history.history['loss'],label="Loss")
plt.plot(model.history.history['auroc'],label="acc")
plt.legend()

In [ ]:
plt.plot(model.history.history['acc'])

In [ ]:
plt.plot(model.history.history['auroc'])

# Utils

## Clean Memory

In [ ]:
del model

In [ ]:
import gc
gc.collect()

# Prediction

In [ ]:
# print('Cargando datos del TEST')
# path = '../../data/test_final_3'
# allFiles = glob.glob(path + "/*.csv")
# list_ = []
# for file_ in allFiles:
#     df = pd.read_csv(file_)
#     df = (df.fillna(-1)).astype(schema_train_3)
#     list_.append(df)


# test = pd.concat(list_, axis = 0, ignore_index = True).fillna(-1)

# sel_cols = [c for c in test.columns if c not in ['MachineIdentifier',
#                                                  'HasDetections',
#                                                  'Census_DeviceFamily_Windows.Server',
#                                                  'Census_DeviceFamily_Windows.Desktop'
#                                                  ]
#             ]

# X_test = test.loc[:, sel_cols]

In [ ]:
# predictions = model.predict(X_test)

In [ ]:
# predf= pd.DataFrame(predictions,columns=['HasDetections'])

In [ ]:
# pd.concat([X_machines,predf],axis=1).to_csv('keras_model.csv', index=None)

In [ ]:
# X_machines = test.loc[:, 'MachineIdentifier']


# df_prds = pd.DataFrame({'MachineIdentifier': X_machines, 'HasDetections': predictions})

# df_prds.to_csv('keras_model.csv', index=None)